# Gerrymandering and the Redistricting Problem

## Assumptions

We assume a 2-party system with the parties represented as "D" and "R". We start by only using one period's worth of historical data. 

## Data Inputs

The expected number of votes for each party is a key data input for our constraints. We represent this with a matrix, $\mathbf{V}$. Since we have two parties, the matrix has the following shape.

$$ \mathbf{V} \in \mathbb{R}^{|blocks| \times 2} $$

$blocks$ is a set where $|blocks|$ is the number of elements in the set. By convention, the first column of $\mathbf{V}$ will be D votes and the second column R votes. $\mathbf{V}$ could represent a single past election's results, or it could be an expected upcoming result based on an exogenous model.

## Variables

We represent the variables with a matrix. Each row of the matrix represents an indivisible block (precint, county, or census area depending on the conventions of the problem). The columns represent assignment to a district. The matrix is made up of zeroes or ones, and each row must have exactly one entry equal to one, meaning that each row must be in one and only one district. $districts$ is a set where $|districts|$ is the number of elements in each set. 

$$ 
\mathbf{D} \in \{0,1\}^{|blocks| \times |districts|}. 
$$

For ease of interpretation, we're going to define another variable that represents the "efficiency gap" in votes. To do this, we need to know the number of expected votes for each party in each block and the assignment of blocks to districts. $\mathbf{D}^T \mathbf{V} \in \mathbb{R}^{|districts| \times 2} $ gives us a matrix of the number of D and R votes in each district. The efficiency gap can be calculated from the seat margin and vote margin. [Brennan Center](https://www.brennancenter.org/sites/default/files/legal-work/How_the_Efficiency_Gap_Standard_Works.pdf)

$$ \text{Efficiency Gap} = (\text{Seat Margin} - 50\%) - 2 ( \text{Vote Margin} - 50\% ) $$

The vote margin is easily calculated by multiplying $\mathbf{D^TV}$ by a vector of ones, $\mathbf{j}$, and subtracting the elements. The seat margin is trickier, since we need to determine which of the two parties is the winner for any given potential solution.  We do this by introducing another vector of variables $\mathbf{w}\in \{0,1\}^{|districts|}$, indicating if D won the given seat (or district). 

$$ \mathbf{w}\in \{0,1\}^{|districts|} $$

When we include $w$ in our objective with an appropriately large penalty, the optimization problem will set $w_i = 1$ if D won the seat and $0$ otherwise. Our constraints will ensure this. So the efficiency gap variable is now:

$$ \text{Efficiency Gap} = \frac{ (\mathbf{w}\cdot \mathbf{j} - (1-\mathbf{w})\cdot \mathbf{j} )}{|districts| } - 0.5 - 2 \left( \frac{\mathbf{j^TD^TV} \left[ \begin{array}{c} 1 \\ -1 \end{array} \right]}{\mathbf{j^TD^TVj}} - 0.5 \right)  $$


## Constraints

To force the $\mathbf{w}$ variable, set a constraint (vector valued):
$$ \mathbf{D^TV} \left[ \begin{array}{c} 1 \\ -1 \end{array} \right] \geq \mathbf{w} $$. 




In [60]:
v = [1 2 3; 3 2 1; 4 5 2]
votes = reshape(votes, prod(size(votes)) )
print(votes)
pop = [73.0; 82.0; 22.0; 9.0; 30.0; 40.0; 26.0; 46.0; 99]
print( pop )
ctg =
[ 1 1 0 1 0 0 0 0 0 ; 
  1 1 1 0 1 0 0 0 0 ;
  0 1 1 0 0 1 0 0 0 ;
  1 0 0 1 1 0 1 0 0 ; 
  0 1 0 1 1 1 0 1 0 ;
  0 0 1 0 1 1 0 0 1 ;
  0 0 0 1 0 0 1 1 0 ; 
  0 0 0 0 1 0 1 1 1 ;
  0 0 0 0 0 1 0 1 1 ]

[1, 3, 4, 2, 2, 5, 3, 1, 2][73.0, 82.0, 22.0, 9.0, 30.0, 40.0, 26.0, 46.0, 99.0]

9×9 Array{Int64,2}:
 1  1  0  1  0  0  0  0  0
 1  1  1  0  1  0  0  0  0
 0  1  1  0  0  1  0  0  0
 1  0  0  1  1  0  1  0  0
 0  1  0  1  1  1  0  1  0
 0  0  1  0  1  1  0  0  1
 0  0  0  1  0  0  1  1  0
 0  0  0  0  1  0  1  1  1
 0  0  0  0  0  1  0  1  1

In [8]:
using JuMP, GLPKMathProgInterface, NLopt

nd = 2
np = length(votes)

#m = Model(solver = GLPKSolverMIP())
m = Model(solver = NLoptSolver())

@variable(m, 0 <= d[i=1:np] <= nd - 1 , Int)
@variable(m, meansize)

# equal size constraint
@constraint(m, sum(pop)  - sum( pop .* d ) <= 1.1 * meansize )
@constraint(m, (1/1.1) * meansize <= sum(pop)  - sum( pop .* d ) )
@constraint(m, sum( pop .* d ) <= 1.1 * meansize )
@constraint(m, (1/1.1) * meansize <=  sum( pop .* d ) )

a = ones(np)
#contiguity constraint
@constraint(m, ( (d .* d') .* ctg ) * a .>= 2 )


@objective(m, Max, sum( p .* d ) )


LoadError: [91mUndefVarError: votes not defined[39m

In [74]:
@time begin
    status = solve(m)
end
println("objective: ", getobjectivevalue(m))
districts = getvalue(d)
reshape(districts, 3,3)

LoadError: [91mSolver does not support quadratic constraints[39m

In [59]:


ctg =
[ 1 1 0 1 0 0 0 0 0 ; 
  1 1 1 0 1 0 0 0 0 ;
  0 1 1 0 0 1 0 0 0 ;
  1 0 0 1 1 0 1 0 0 ; 
  0 1 0 1 1 1 0 1 0 ;
  0 0 1 0 1 1 0 0 1 ;
  0 0 0 1 0 0 1 1 0 ; 
  0 0 0 0 1 0 1 1 1 ;
  0 0 0 0 0 1 0 1 1 ]
ctg * (districts .== 1 )


9-element Array{Int64,1}:
 2
 3
 3
 3
 4
 3
 2
 2
 1

In [57]:
reshape(votes,3,3)



3×3 Array{Int64,2}:
 1  2  3
 3  2  1
 4  5  2

In [11]:
A = [75 25; 60 40; 43 57; 48 52; 49 51]

DR_totals = ones(1,5) * A
V_margin = DR_totals[1,1] - DR_totals[1,2]
min.(A)

5×2 Array{Int64,2}:
 75  25
 60  40
 43  57
 48  52
 49  51